<a href="https://colab.research.google.com/github/Zeddling/BDSS/blob/main/scripts/model_bdss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install flask-restful

     |████████████████████████████████| 51kB 3.9MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Imports

In [ ]:
import cv2
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
from flask_restful import Resource, Api, reqparse
import numpy as np
import tensorflow as tf
from tensorflow import keras
import werkzeug

In [ ]:
app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)   #starts ngrok when the app is run

In [ ]:
def gray_scale(img):
  return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [ ]:
def threshold(img):
  kernel = np.ones((3,10), np.uint8)
  blur = cv2.GaussianBlur(img, (5,5), 0)
  img_thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
  img_dilate = cv2.dilate(img_thresh, kernel, iterations=1)
  img_erode = cv2.erode(img_dilate, kernel, iterations=1)
  return cv2.medianBlur(img_erode, 7)

In [ ]:
def make_clearer(img):
  blur = cv2.GaussianBlur( img, (5,5), 0 )
  clahe = cv2.createCLAHE(clipLimit=5)
  clear_img = clahe.apply(img) + 30
  return cv2.cvtColor(clear_img, cv2.COLOR_GRAY2RGB)

In [ ]:
def mask_and_segment(img, original_image):
  cnt = sorted(cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key=cv2.contourArea)[-1]
  mask = np.zeros((227, 227), np.uint8)
  cv2.drawContours(mask, [cnt], -1, 255, -1)
  dst = cv2.bitwise_and(original_image, original_image, mask=mask)
  segmented_img = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
  return make_clearer(segmented_img)

In [ ]:
import os

def process_single_image(img_file):
  output_dir = "/content/clean_"+os.path.basename(img_file)
  print(output_dir)
  img_mat = cv2.imread(img_file)
  img = cv2.resize( img_mat, (227, 227), interpolation=cv2.INTER_AREA )
  gray = gray_scale(img)
  thresh = threshold(gray)
  segmented = mask_and_segment(thresh, img)
  cv2.imwrite(output_dir, segmented)
  return output_dir

In [ ]:
def AlexNet():
  return tf.keras.models.load_model("/content/drive/MyDrive/BDSS/clahe_equalized_split/alexnet_v3.h5")

In [ ]:
def infer(filepath):
  print("Making inferrence")
  class_names = ['benign', 'malignant', 'normal']
  img_path = process_single_image(filepath)
  print(img_path)
  img = cv2.imread(img_path)
  np_img_data = np.asarray(img)
  rescale_img_data = cv2.normalize(np_img_data.astype('float'), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  img_array = np.expand_dims(rescale_img_data, axis=0)
  print("Predicting......{}" )
  model = AlexNet()
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  pred = score.numpy()
  print(score)
  return jsonify({
      "class_name": class_names[np.argmax(score)],
      "benign": 100 * pred[0],
      "malignant": 100 * pred[1],
      "normal": 100 * pred[2]
  })

In [ ]:
class ____Inference___(Resource):
  def post(self):
    parse = reqparse.RequestParser()
    parse.add_argument('file', type=werkzeug.datastructures.FileStorage, location='files')
    args = parse.parse_args()
    image = args['file']
    output_dir = "/content/"+image.filename
    image.save(output_dir)
    return infer(output_dir)

api.add_resource(____Inference___, "/infer")
if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9b0f340bb7d4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Making inferrence
/content/clean_20586934 (6).png
/content/clean_20586934 (6).png
Predicting......{}


127.0.0.1 - - [10/Feb/2021 07:39:22] "POST /infer HTTP/1.1" 200 -


tf.Tensor([0.21194185 0.5761154  0.21194272], shape=(3,), dtype=float32)
Making inferrence
/content/clean_20586934.png
/content/clean_20586934.png
Predicting......{}


127.0.0.1 - - [10/Feb/2021 08:02:10] "POST /infer HTTP/1.1" 200 -


tf.Tensor([0.21194316 0.5761093  0.21194758], shape=(3,), dtype=float32)
